In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import time
from helper_functions.sketching import *

data = pd.read_csv('./data/capture20110811.pcap.netflow.labeled', sep ='\s+')
data.head()

,#Date_flow,start,Durat,Prot,Src_IP_Addr:Port,->,Dst_IP_Addr:Port,Flags,Tos,Packets,Bytes,Flows,Label(LEGITIMATE:Botnet:Background)
0,2011-08-11,10:10:00.003,0.967,UDP,89.31.8.11:23929,->,147.32.84.229:13363,INT,0,2,135,1,Background
1,2011-08-11,10:10:00.003,0.967,UDP,147.32.84.229:13363,->,89.31.8.11:23929,INT,0,2,276,1,Background
2,2011-08-11,10:10:00.006,0.000,UDP,208.88.186.6:34042,->,147.32.84.229:13363,INT,0,1,62,1,Background
3,2011-08-11,10:10:00.008,0.000,UDP,92.118.218.77:55246,->,147.32.84.229:13363,INT,0,1,78,1,Background
4,2011-08-11,10:10:00.009,0.000,UDP,182.185.139.181:10223,->,147.32.84.229:13363,INT,0,1,72,1,Background


In [2]:
data[['Src_IP_Addr', 'Src_Port']] = data['Src_IP_Addr:Port'].str.split(':', n=1, expand=True)
data[['Dst_IP_Addr', 'Dst_Port']] = data['Dst_IP_Addr:Port'].str.split(':', n=1, expand=True)

data['Label'] = data['Label(LEGITIMATE:Botnet:Background)']

data = data.drop(labels=['->', 'Label(LEGITIMATE:Botnet:Background)', 'Src_IP_Addr:Port', 'Dst_IP_Addr:Port'], axis=1)

data.head()

,#Date_flow,start,Durat,Prot,Flags,Tos,Packets,Bytes,Flows,Src_IP_Addr,Src_Port,Dst_IP_Addr,Dst_Port,Label
0,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,135,1,89.31.8.11,23929,147.32.84.229,13363,Background
1,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,276,1,147.32.84.229,13363,89.31.8.11,23929,Background
2,2011-08-11,10:10:00.006,0.000,UDP,INT,0,1,62,1,208.88.186.6,34042,147.32.84.229,13363,Background
3,2011-08-11,10:10:00.008,0.000,UDP,INT,0,1,78,1,92.118.218.77,55246,147.32.84.229,13363,Background
4,2011-08-11,10:10:00.009,0.000,UDP,INT,0,1,72,1,182.185.139.181,10223,147.32.84.229,13363,Background


In [3]:
infected_host = '147.32.84.165'

data_infected = data[data['Src_IP_Addr']==infected_host]
dst_hosts = data_infected['Dst_IP_Addr'].values
unique_dst_hosts = np.unique(dst_hosts)

print("Number of hosts connected with infected: ", len(unique_dst_hosts))

ground_truth = data_infected.groupby(by='Dst_IP_Addr').size()
ground_truth = ground_truth.sort_values(ascending=False)
ground_truth.head(10)

Number of hosts connected with infected:  1662


Dst_IP_Addr
193.23.181.44      6167
174.128.246.102    3427
174.37.196.55      3357
67.19.72.206       3135
72.20.15.61        2966
173.236.31.226     1708
184.154.89.154     1678
46.4.36.120        1627
147.32.80.9         787
217.163.21.37       701
dtype: int64

In [23]:
start_time = time.time()
cm, seeds = construct_cm(dst_hosts, 20, 50)
estim_count = estimate_counts(unique_dst_hosts, cm, seeds)
sorted_ind = np.flip(np.argsort(estim_count))
dur = time.time() - start_time
print("Duration in seconds: ", dur)

Duration in seconds:  0.7244007587432861


In [24]:
d = {'Dst_IP_Addr':unique_dst_hosts[sorted_ind], 'Counts':estim_count[sorted_ind]}
dd = pd.DataFrame(data=d)
dd.head(10)

,Dst_IP_Addr,Counts
0,193.23.181.44,6236
1,174.128.246.102,3498
2,174.37.196.55,3489
3,67.19.72.206,3209
4,72.20.15.61,3033
5,173.236.31.226,1801
6,46.4.36.120,1756
7,184.154.89.154,1746
8,147.32.80.9,877
9,217.163.21.37,766


In [15]:
depths = np.array([5, 10, 20, 50], dtype=int)
widths = np.array([10, 100, 1000])

start_time = time.time()
dic = {}

i = 0
for d in depths:
    for w in widths:
        cm, seeds = construct_cm(dst_hosts, d, w)
        estim_count = estimate_counts(unique_dst_hosts, cm, seeds)
        sorted_ind = np.flip(np.argsort(estim_count))
        
        dic[i] = {'Dst_IP_Addr':unique_dst_hosts[sorted_ind], 'Counts':estim_count[sorted_ind]}
        i = i+1
        
dur = time.time() - start_time
print("Duration in seconds: ", dur)

Duration in seconds:  10.986593246459961


In [23]:
df = pd.DataFrame(data=dic[1])
df.head(10)

,Dst_IP_Addr,Counts
0,193.23.181.44,6198
1,174.128.246.102,3481
2,174.37.196.55,3420
3,67.19.72.206,3272
4,72.20.15.61,2999
5,184.154.89.154,1758
6,173.236.31.226,1754
7,46.4.36.120,1679
8,147.32.80.9,809
9,217.163.21.37,747
